In [ ]:
import tensorflow as tf
print(tf.__version__)
import sys
print (sys.version)
import numpy as np
import time
import math as mt
import matplotlib.pyplot as plt

In [ ]:
StartingValuesFromFile=np.load('startingvaluesMultiVariatePoisson.npy')
print(StartingValuesFromFile.shape)


Now calculate the outputs...

In [ ]:
YFromFile=np.load('outputsMultiVariatePoisson.npy')
print(YFromFile)

In [ ]:
import os
os.getcwd()

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Lambda
from tensorflow.keras.losses import MeanSquaredError
#from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import initializers
from tensorflow.keras.layers import Input, Add
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K

In [ ]:
#Custom activation function
vector_z=[1.,1.,1.,1.,1.,1.,1.,1.,1.,1.] #This is the vector z we need to choose in Example 2.12
vector_psi=[0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25] #This is the vector that encodes the function psi in Example 2.12
psi=tf.constant(vector_psi)
z=tf.constant(vector_z)   

def custom_fct(x):
    psi_mat=tf.expand_dims(psi, axis=0)
    psi_mat_t=tf.transpose(psi_mat)
    #psi_mat_batch=tf.broadcast_to(psi_mat, [3,1,3])
    tmp=tf.matmul(x,psi_mat_t)
    tmp2=K.maximum(tf.zeros_like(tmp),1-K.exp(-tmp))  
    tmp3=tmp2*z
    return tmp3#tf.math.scalar_mul(tmp4, z)

In [ ]:
approxDim=10 #This is the approximation dimension, the number of basis functions we consider

def genNetwork2Layer(NbNodesPerLayer):
    inputs =  tf.keras.Input(shape=(approxDim,))

    nodes1 = [] 
    nodes2 = []
    #After each node is a lambda layer that implements our activation function
    lambda_layers1= []
    lambda_layers2= []
    #Final linear forms that are applied
    linear_form_layers=[]
    all_linear_form_outputs=[]
    for j in range(NbNodesPerLayer):
        node1 = Dense(approxDim, activation=None,trainable=True,
                    kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                    bias_initializer='random_normal',
                    name=str('node1_p')+str('_k')+str(k)+str('_j')+str(j))
        lamb1= Lambda(custom_fct,name=str('activation_1_p')+str('_k')+str(k)+str('_j')+str(j),output_shape=(approxDim,))
        nodes1 = nodes1 + [node1]
        
        lambda_layers1=lambda_layers1 + [lamb1]
        
        node2 = Dense(approxDim, activation=None,trainable=True,
                    kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                    bias_initializer='random_normal',
                    name=str('node2_p')+str('_k')+str(k)+str('_j')+str(j))
        lamb2= Lambda(custom_fct,name=str('activation_2_p')+str('_k')+str(k)+str('_j')+str(j),output_shape=(approxDim,))
        
        nodes2 = nodes2 + [node2]
        lambda_layers2=lambda_layers2 + [lamb2]
        
        lin= Dense(1, activation=None,trainable=True,
                    kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                    use_bias=False,
                    name=str('linearForm_p')+str('_k')+str(k)+str('_j')+str(j))

        linear_form_layers=linear_form_layers + [lin]

        output_linearmap1=nodes1[j](inputs)
        output_activation1=lambda_layers1[j](output_linearmap1)
        output_linearmap2=nodes2[j](output_activation1)
        output_activation2=lambda_layers2[j](output_linearmap2)
        output_linear_form=linear_form_layers[j](output_activation2)
            
        all_linear_form_outputs=all_linear_form_outputs+[output_linear_form]
    

    #print(all_linear_form_outputs)  
        add_layer=Add(name=str('addlayer_p')+str('_k')+str(k))

    if NbNodesPerLayer>1:
        finalOutput=add_layer(all_linear_form_outputs)
        model = Model(inputs=inputs, outputs=finalOutput,name=str('model_p')+str('_k')+str(k))            
    else:
        model = Model(inputs=inputs, outputs=output_linear_form,name=str('model_p')+str('_k')+str(k))  

    return model

In [ ]:
NbDataPartitions=5


errors=np.zeros(NbDataPartitions)
for k in range(15,20,1):
    ticall=time.perf_counter()
    for p in range(0,NbDataPartitions,1):
    #for p in range(0,1,1):
        StartingValues=np.concatenate((StartingValuesFromFile[:p*1000000], StartingValuesFromFile[(p+1)*1000000:]), axis=0)
        Y=np.concatenate((YFromFile[:p*1000000], YFromFile[(p+1)*1000000:]), axis=0)
        tic = time.perf_counter()
        model=genNetwork2Layer(k)
        model.compile(optimizer='adam',loss='mean_squared_error')
        model.fit(StartingValues, Y, epochs=25, batch_size=10000)
        errors[p]=model.evaluate(StartingValuesFromFile[p*1000000:(p+1)*1000000],YFromFile[p*1000000:(p+1)*1000000])
        toc = time.perf_counter()
        print(f"Calc time: {(toc - tic)/60.0:0.4f} minutes")

    filename=str('error_model_2Layer')+str(k)
    np.save(filename,errors)
    tocall=time.perf_counter()
    print(f"Calc time: {(tocall - ticall)/60.0:0.4f} minutes")
    

In [ ]:
avgerror=np.zeros(19)
for k in range(1,20,1):
    error=np.load('error_model_2Layer'+str(k)+'.npy')
    avgerror[k-1]=np.average(error)

print(avgerror)
filename=str('avg_error_2layer')
np.save(filename,avgerror)

bestmodel2Layer=np.argmin(avgerror)

avgerror1Layer=np.load('avg_error_1layer.npy')
bestmodel1Layer=np.argmin(avgerror1Layer)
print(bestmodel1Layer)

fig=plt.figure()
ax=plt.axes()
xvalues=range(1,20,1)
#plt.plot(xvalues,avgerror1Layer,avgerror)
ax.set(xlabel='number of nodes in hidden layer', ylabel='avg mean squared error',
       title='')


ax.plot(xvalues,avgerror1Layer,markevery=[bestmodel1Layer], marker="o",label='1-layer')
ax.plot(xvalues,avgerror,markevery=[bestmodel2Layer], marker="o",label='2-layer')
plt.legend(loc='upper right')
plt.show()

In [ ]:
#model=genNetwork2Layer(2)
#model.compile(optimizer='adam',loss='mean_squared_error')
#StartingValues=np.concatenate((StartingValuesFromFile[:1*1000000], StartingValuesFromFile[(1+1)*1000000:]), axis=0)
#Y=np.concatenate((YFromFile[:1*1000000], YFromFile[(1+1)*1000000:]), axis=0)
#model.fit(StartingValues, Y, epochs=25, batch_size=10000)
#tmperror=model.evaluate(StartingValuesFromFile[1*1000000:(1+1)*1000000],YFromFile[1*1000000:(1+1)*1000000])
#error=np.load('error_model_2Layer'+str(2)+'.npy')
#print(error)
#error[1]=tmperror
#filename=str('error_model_2Layer2')
#np.save(filename,error)

In [ ]:
model=genNetwork2Layer(bestmodel2Layer)
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(StartingValuesFromFile, YFromFile, epochs=25, batch_size=10000)

In [ ]:
tmp=np.load('startingvaluesMultiVariatePoissonKoefficientTestset100000Sim0.npy')
print(tmp.shape)

In [ ]:
#MCpricesTestset=np.load('outputsMultiVariateTestset.npy')
#xvaluesTestset=np.load('startingvaluesMultiVariateTestset.npy')
startingValuesTestset=np.zeros([960,10])
MCprices=np.zeros([960])
for k in range(0,15,1):
    tmpstring="% s" % k
    filename1="startingvaluesMultiVariatePoissonKoefficientTestset100000Sim"+tmpstring+".npy"
    filename2="outputsMultiVariatePoissonKoefficientTestset100000Sim"+tmpstring+".npy"
    tmp1=np.load(filename1)
    tmp2=np.load(filename2)
    startingValuesTestset[k*64:(k+1)*64]=tmp1
    MCprices[k*64:(k+1)*64]=tmp2  

In [ ]:
mctf=tf.reshape(tf.constant(MCprices),[MCprices.shape[0],1])
mctf=tf.dtypes.cast(mctf, tf.float32)
mse = tf.keras.losses.MeanSquaredError()

In [ ]:
#Train all other networks and calc error
mctf=tf.reshape(tf.constant(MCprices),[MCprices.shape[0],1])
mctf=tf.dtypes.cast(mctf, tf.float32)
mse = tf.keras.losses.MeanSquaredError()
errorsModels=np.zeros(19)
for k in range(1,20,1):
    model=genNetwork2Layer(k)
    model.compile(optimizer='adam',loss='mean_squared_error')
    model.fit(StartingValuesFromFile, YFromFile, epochs=25, batch_size=10000)
    predictions=model(startingValuesTestset)
    errorsModels[k-1]=mse(predictions, mctf).numpy()
    
np.save('error2LayerModels',errorsModels)   
    

In [ ]:
error1LayerModels=np.load('error1LayerModels.npy')
errorsModels=np.load('error2LayerModels.npy')
print(error1LayerModels)

best1Layer=np.argmin(error1LayerModels)
best2Layer=np.argmin(errorsModels)

print(error1LayerModels[best1Layer])

print(errorsModels[best2Layer])

fig=plt.figure()
ax=plt.axes()
xvalues=range(1,20,1)
#plt.plot(xvalues,avgerror1Layer,avgerror)
ax.set(xlabel='number of nodes in hidden layer', ylabel='avg mean squared error',
       title='')


ax.plot(xvalues,error1LayerModels,markevery=[best1Layer], marker="o",label='1-layer')
ax.plot(xvalues,errorsModels,markevery=[best2Layer], marker="o",label='2-layer')
plt.legend(loc='upper right')
plt.show()